In [1]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install "git+https://github.com/albumentations-team/albumentations.git"

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-u4kiogyc
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-u4kiogyc
     |████████████████████████████████| 106 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 13.6 MB/s eta 0:00:01
  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=7338214 sha256=cd21645f934dfebeb2c8b38216ec544d6e02531400c602ec436a0ca26837f46a
  Stored in directory: /tmp/pip-ephem-wheel-cache-xi72arzr/wheels/19/ac/65/e48e5e4ec2702274d927c5a6efb75709b24014371d3bb778f2
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60943 sha25

In [1]:
import sys
sys.path.append("utils")

In [2]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer
from albumentations import *
import torch
import os
from detectron2.data import detection_utils
from utils.aug import MyMapper
from detectron2.solver.build import *
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import opendatasets as od
import warnings
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
class Config:
  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]
  anchor_generators_sizes = [[8], [16], [32], [64],[128]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]

In [4]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

#     @classmethod
#     def build_train_loader(cls, cfg, sampler=None):
#         return build_detection_train_loader(
#             cfg, mapper=MyMapper(cfg), sampler=sampler
#         )

    def build_hooks(self):
      # copy of cfg
      cfg = self.cfg.clone()

      # build the original model hooks
      hooks = super().build_hooks()

      # add the best checkpointer hook
      hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
                                        DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                                        "MaP IoU",
                                        "max",
                                        ))
      return hooks

    # @classmethod
    # def build_optimizer(cls,cfg,model) -> torch.optim.Optimizer:
    #   """
    #   Build an optimizer from config.
    #   """
    #   params = get_default_optimizer_params(
    #       model,
    #       base_lr=cfg.SOLVER.BASE_LR,
    #       weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
    #       bias_lr_factor=cfg.SOLVER.BIAS_LR_FACTOR,
    #       weight_decay_bias=cfg.SOLVER.WEIGHT_DECAY_BIAS,
    #   )
    #   return maybe_add_gradient_clipping(cfg, torch.optim.SGD)(
    #       params,
    #       lr=cfg.SOLVER.BASE_LR,
    #       momentum=cfg.SOLVER.MOMENTUM,
    #       nesterov=cfg.SOLVER.NESTEROV,
    #       weight_decay=cfg.SOLVER.WEIGHT_DECAY,
    #   )

    # @classmethod
    # def build_lr_scheduler(cls,cfg, optimizer) -> torch.optim.lr_scheduler._LRScheduler:
    #   """
    #   Build a LR scheduler from config.
    #   """
    #   name = cfg.SOLVER.LR_SCHEDULER_NAME

    #   if name == "WarmupMultiStepLR":
    #       steps = [x for x in cfg.SOLVER.STEPS if x <= cfg.SOLVER.MAX_ITER]
    #       if len(steps) != len(cfg.SOLVER.STEPS):
    #           logger = logging.getLogger(__name__)
    #           logger.warning(
    #               "SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. "
    #               "These values will be ignored."
    #           )
    #       sched = MultiStepParamScheduler(
    #           values=[cfg.SOLVER.GAMMA ** k for k in range(len(steps) + 1)],
    #           milestones=steps,
    #           num_updates=cfg.SOLVER.MAX_ITER,
    #       )
    #   elif name == "WarmupCosineLR":
    #       sched = CosineParamScheduler(1, 1/40)
    #   elif name == "CyclicLR":
    #       sched = torch.optim.lr_scheduler.CyclicLR(optimizer,
    #                                                 base_lr=cfg.SOLVER.BASE_LR, max_lr=5*cfg.SOLVER.BASE_LR,
    #                                                 base_momentum=0.85, max_momentum=0.95)

    #   else:
    #       raise ValueError("Unknown LR scheduler: {}".format(name))

    #   sched = WarmupParamScheduler(
    #       sched,
    #       cfg.SOLVER.WARMUP_FACTOR,
    #       min(cfg.SOLVER.WARMUP_ITERS / cfg.SOLVER.MAX_ITER, 1.0),
    #       cfg.SOLVER.WARMUP_METHOD,
    #   )
    #   return LRMultiplier(optimizer, multiplier=sched, max_iter=cfg.SOLVER.MAX_ITER)

In [5]:
def run():
  dataDir = "data"
  DatasetCatalog.clear()
  MetadataCatalog.clear()
  register_coco_instances(f'sartorius_train',{}, 'input/all/annotations_train.json'.format(fold), dataDir)
  register_coco_instances(f'sartorius_val',{},'input/all/annotations_val.json'.format(fold), dataDir)

  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
  cfg.DATASETS.TRAIN = (f"sartorius_train",)
  cfg.DATASETS.TEST = (f"sartorius_val",)
  # cfg.MODEL.PIXEL_MEAN = Config.pixel_mean
  # cfg.MODEL.PIXEL_STD = Config.pixel_std

  cfg.DATALOADER.NUM_WORKERS = 7
  cfg.MODEL.WEIGHTS = "pretrained/final_weapon_without_copy/model_0029999.pth"#"/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/LIVECell_anchor_based_model.pth"
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 1e-3
  cfg.SOLVER.WEIGHT_DECAY = 5e-5
  cfg.SOLVER.WARMUP_ITERS = 200
  #cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
  cfg.SOLVER.MAX_ITER = 10000 #len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH * 20 #尝试20ep 用lr调整
  cfg.SOLVER.STEPS = (7000,9000,9500)
  #cfg.SOLVER.GAMMA = 0.2
  cfg.INPUT.MIN_SIZE_TRAIN = (1000,1100,1200,1300)
  cfg.INPUT.MAX_SIZE_TRAIN = 2000
  cfg.INPUT.CROP.ENABLE = False
  cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
  cfg.INPUT.MIN_SIZE_TEST = 1300
  cfg.INPUT.MAX_SIZE_TEST = 2000
  cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  #cfg.TEST.EVAL_PERIOD = 
  cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  cfg.SOLVER.AMP.ENABLED  = False
  cfg.SEED = 42

#   cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = Config.anchor_generators_aspect_ratios
#   cfg.MODEL.ANCHOR_GENERATOR.SIZES = Config.anchor_generators_sizes
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # #cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
  # cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS = (0.5, 0.6, 0.7)

  cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  
    
  # cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_WEIGHT = 10.0
  #cfg.SOLVER.AMP.ENABLED = True

  cfg.OUTPUT_DIR = "pretrained/final_weapon_without_copy/finetuned/".format(fold)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [ ]:
# 2 的理论score最高
for fold in range(1,2):
  run()

#run_all()

[12/29 02:18:32 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (11, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (11,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (11, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (11,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.2.cls_score.weight' to the model due to incompatible shapes: (11, 1024) in the checkpoint but (4, 1024) 

[12/29 02:18:35 d2.engine.train_loop]: Starting training from iteration 0


[W LegacyTypeDispatch.h:74] Warning: AutoNonVariableTypeMode is deprecated and will be removed in 1.10 release. For kernel implementations please use AutoDispatchBelowADInplaceOrView instead, If you are looking for a user facing API to enable running your inference-only workload, please use c10::InferenceMode. Using AutoDispatchBelowADInplaceOrView in user code is under risk of producing silent wrong result in some edge cases. See Note [AutoDispatchBelowAutograd] for more details. (function operator())


[12/29 02:19:33 d2.utils.events]:  eta: 6:09:57  iter: 19  total_loss: 7.446  loss_cls_stage0: 1.308  loss_box_reg_stage0: 0.4966  loss_cls_stage1: 1.286  loss_box_reg_stage1: 0.8747  loss_cls_stage2: 1.3  loss_box_reg_stage2: 0.8636  loss_mask: 0.6821  loss_rpn_cls: 0.4162  loss_rpn_loc: 0.262  time: 2.1819  data_time: 0.6252  lr: 9.5905e-05  max_mem: 24696M
[12/29 02:20:17 d2.utils.events]:  eta: 6:08:03  iter: 39  total_loss: 5.564  loss_cls_stage0: 0.7845  loss_box_reg_stage0: 0.4847  loss_cls_stage1: 0.8014  loss_box_reg_stage1: 0.8118  loss_cls_stage2: 0.785  loss_box_reg_stage2: 0.8589  loss_mask: 0.5508  loss_rpn_cls: 0.2279  loss_rpn_loc: 0.2505  time: 2.1916  data_time: 0.0746  lr: 0.0001958  max_mem: 24709M
[12/29 02:21:01 d2.utils.events]:  eta: 6:10:27  iter: 59  total_loss: 4.691  loss_cls_stage0: 0.5868  loss_box_reg_stage0: 0.4691  loss_cls_stage1: 0.5752  loss_box_reg_stage1: 0.8431  loss_cls_stage2: 0.546  loss_box_reg_stage2: 0.9057  loss_mask: 0.4305  loss_rpn_cls: 